In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./vaswani2017-AttentionIsAllYouNeed.pdf")
docs = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)


In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents[:30], OllamaEmbeddings(model="mymerak"))

C:\Users\HP\AppData\Local\Temp\ipykernel_19060\2866218600.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(documents[:30], OllamaEmbeddings(model="mymerak"))


In [4]:
query = "Apa itu Attention Mechanism"
result = db.similarity_search(query)
print(result[0].page_content)

√dk
)V (1)
The two most commonly used attention functions are additive attention [2], and dot-product (multi-
plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor
of 1√dk
. Additive attention computes the compatibility function using a feed-forward network with
a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is
much faster and more space-efficient in practice, since it can be implemented using highly optimized
matrix multiplication code.
While for small values of dk the two mechanisms perform similarly, additive attention outperforms
dot product attention without scaling for larger values of dk [3]. We suspect that for large values of
dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has
extremely small gradients 4. To counteract this effect, we scale the dot products by 1√dk
.
3.2.2 Multi-Head Attention


In [5]:
from langchain_community.llms import Ollama

llm = Ollama(model="mymerak")
llm

C:\Users\HP\AppData\Local\Temp\ipykernel_19060\3725745729.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mymerak")


Ollama(model='mymerak')

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Jawablah pertanyaan saya hanya berdasarkan konteks yang saya berikan.
Berfikirlah langkah demi langkah sebelum memberikan jawaban secara detail.
<context> 
{context}
</context>
Question: {input}                                 
""")

In [7]:
# Chain / Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [8]:
# retriever
retriever = db.as_retriever()

In [9]:
# retrieval chain
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
retrieval_chain.invoke({
    "input": "Apa itu Attention Mechanism"
})

{'input': 'Apa itu Attention Mechanism',
 'context': [Document(metadata={'source': './vaswani2017-AttentionIsAllYouNeed.pdf', 'page': 3}, page_content='√dk\n)V (1)\nThe two most commonly used attention functions are additive attention [2], and dot-product (multi-\nplicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor\nof 1√dk\n. Additive attention computes the compatibility function using a feed-forward network with\na single hidden layer. While the two are similar in theoretical complexity, dot-product attention is\nmuch faster and more space-efficient in practice, since it can be implemented using highly optimized\nmatrix multiplication code.\nWhile for small values of dk the two mechanisms perform similarly, additive attention outperforms\ndot product attention without scaling for larger values of dk [3]. We suspect that for large values of\ndk, the dot products grow large in magnitude, pushing the softmax function into regions wher

In [12]:
result = retrieval_chain.invoke({
    "input": "Apa itu Attention Mechanism"
})

print(result["answer"])

Attention adalah kemampuan untuk menekankan atau menarik perhatian pada bagian tertentu dari input yang relevan, membuatnya lebih jelas untuk dianalisis. Ini merupakan konsep psikologi dan sosiologi yang melibatkan fokus kesadaran atau perhatian pada stimulus tertentu dengan mengabaikan atau meminimalkan kehadiran yang lain. Contohnya, ketika kita sedang berbicara dengan seseorang, kita fokus pada apa yang mereka katakan dan menarik perhatian kami pada pesan tertentu dalam percakapan. Kemampuan untuk menarik perhatian dapat membantu mengorganisir dan menyaring informasi, serta meningkatkan kognisi dan empati.

Secara lebih umum, pada tingkat komputer, attention adalah mekanisme yang mengatur perhatian komputer untuk memfokuskan proses pemrosesan informasi pada bagian tertentu dari data. Ini dapat digunakan dalam jaringan saraf, di mana itu menghubungkan neuron yang berbeda untuk membentuk koneksi dan mengurangi sumber daya komputasi. Dalam konteks ini, attention biasanya diterapkan pad

In [16]:
response = retrieval_chain.invoke({
    "input": "Apa saja komponen-komponen dari scaled dot-product attention?"
})

print(response["answer"])

Scaled dot-product attention adalah bagian integral dari model Transformer yang digunakan dalam pemrosesan bahasa alami. Scaled dot-product attention melibatkan tiga operasi kunci: query, key, dan value. Ketiganya merupakan tensor dengan bentuk (T, H), dimana T adalah panjang dari seri masukan dan H adalah dimensi representasi hidden-state yang digunakan dalam model.

Query, key, dan value dihasilkan oleh jaringan saraf sebelumnya pada setiap blok Transformer, dan mereka mewakili informasi yang berbeda tentang seri masukan: Query merupakan perkiraan representasi informasi yang ingin ditemukan dalam seri masukan, sementara Key dan Value mewakili bagian-bagian dari seri masukan yang mungkin relevan.

Scaled dot-product attention dirancang untuk menghitung seberapa erat kaitannya antara query dan key, sambil juga mempertimbangkan panjang dari keduanya. Ini dilakukan dengan mencalculasi produk titik skala dot (dengan skala 1/sqrt(H)) antara query dan key, lalu membaginya dengan jumlah elem

In [17]:
from langchain_core.output_parsers import StrOutputParser
chain_wo_rag = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please response to the user queries"),
        ("user","Question:{question}")
    ]
) | llm | StrOutputParser()

result_without_rag = chain_wo_rag.invoke({"question":"Apa saja komponen-komponen dari scaled dot-product attention?"})

In [18]:
result_with_rag = response["answer"]
print(f"[ANSWER WITH RAG CHAIN]\n{result_with_rag}")
print()
print(f"[ANSWER WITHOUT RAG CHAIN]\n{result_without_rag}")

[ANSWER WITH RAG CHAIN]
Scaled dot-product attention adalah bagian integral dari model Transformer yang digunakan dalam pemrosesan bahasa alami. Scaled dot-product attention melibatkan tiga operasi kunci: query, key, dan value. Ketiganya merupakan tensor dengan bentuk (T, H), dimana T adalah panjang dari seri masukan dan H adalah dimensi representasi hidden-state yang digunakan dalam model.

Query, key, dan value dihasilkan oleh jaringan saraf sebelumnya pada setiap blok Transformer, dan mereka mewakili informasi yang berbeda tentang seri masukan: Query merupakan perkiraan representasi informasi yang ingin ditemukan dalam seri masukan, sementara Key dan Value mewakili bagian-bagian dari seri masukan yang mungkin relevan.

Scaled dot-product attention dirancang untuk menghitung seberapa erat kaitannya antara query dan key, sambil juga mempertimbangkan panjang dari keduanya. Ini dilakukan dengan mencalculasi produk titik skala dot (dengan skala 1/sqrt(H)) antara query dan key, lalu memba

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import ArxivRetriever
from langchain_ollama.chat_models import ChatOllama

# prompt = ChatPromptTemplate.from_template("""
# Jawablah pertanyaan saya hanya berdasarkan konteks yang saya berikan.
# Berfikirlah langkah demi langkah sebelum memberikan jawaban secara detail.
# <context> 
# {context}
# </context>
# Pertanyaan: {input}                                 
# """)

# prompt = ChatPromptTemplate.from_template("""
# Answer following question based only on the provided context.
# Think step by step before providing a detailed answer.
# I will tip you $1000 if the user finds the answer is helpful.
# <context> 
# {context}
# </context>
# Question: {input}                                 
# """)

ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

llm = ChatOllama(model="llama3.2")

def format_docs(docs):
    document = ""
    for doc in docs:
        paper = "\n".join(f"{k}: {v}" for k, v in doc.metadata.items())
        paper = paper + "\n" + doc.page_content + "\n\n"
        document = document + paper        
    return document

arxiv_retriever = ArxivRetriever(load_max_docs=1, get_full_documents=True)

chain = (
    {"context": arxiv_retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
response = chain.invoke("Who is the author of the paper titled SparseSwin: Swin Transformer with Sparse Transformer Block on ArXiv")

In [26]:
response

'The authors of the paper "SparseSwin: Swin Transformer with Sparse Transformer Block" are not explicitly mentioned in the provided text. However, based on the context and the title of the paper, it is likely that the authors are researchers working in the field of computer vision or natural language processing.\n\nIt seems that the question you asked earlier was related to a different paper titled "SPEECH SWIN-TRANSFORMER: EXPLORING A HIERARCHICAL TRANSFORMER WITH SHIFTED WINDOWS FOR SPEECH EMOTION RECOGNITION". The authors of this paper are:\n\n1. Yong Wang\n2. Cheng Lu\n3. Hailun Lian\n4. Yan Zhao\n5. Bj¨orn W. Schuller\n6. Yuan Zong\n7. Wenming Zheng'

In [ ]:
chain.invoke("Siapakah penulis paper dengan judul SparseSwin: Swin Transformer with Sparse Transformer Block")

' Penulis paper yang berjudul "SparseSwin: Swin Transformer dengan Blok Transformator Khusus" adalah Wei Liu, Wenzhu Xing, Karen Egiazarian dan Jiaxin Hu.'

In [9]:
arxiv_retriever.invoke("Siapakah author dari paper SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv")

[Document(metadata={'Published': '2023-09-11', 'Title': 'SparseSwin: Swin Transformer with Sparse Transformer Block', 'Authors': 'Krisna Pinasthika, Blessius Sheldo Putra Laksono, Riyandi Banovbi Putera Irsal, Syifa Hukma Shabiyya, Novanto Yudistira', 'Summary': 'Advancements in computer vision research have put transformer architecture as\nthe state of the art in computer vision tasks. One of the known drawbacks of\nthe transformer architecture is the high number of parameters, this can lead to\na more complex and inefficient algorithm. This paper aims to reduce the number\nof parameters and in turn, made the transformer more efficient. We present\nSparse Transformer (SparTa) Block, a modified transformer block with an\naddition of a sparse token converter that reduces the number of tokens used. We\nuse the SparTa Block inside the Swin T architecture (SparseSwin) to leverage\nSwin capability to downsample its input and reduce the number of initial tokens\nto be calculated. The propose

In [11]:
arxiv_retriever.invoke("Jelaskan saya mengenai Deformable Attention")

[Document(metadata={'Published': '2013-09-10', 'Title': 'Stable numerical coupling of exterior and interior problems for the wave equation', 'Authors': 'Lehel Banjai, Christian Lubich, Francisco-Javier Sayas', 'Summary': 'Stable numerical coupling of exterior and interior problems for the wave\nequation'}, page_content='arXiv:1309.2649v1  [math.NA]  10 Sep 2013\nNoname manuscript No.\n(will be inserted by the editor)\nStable numerical coupling of exterior and interior\nproblems for the wave equation\nLehel Banjai · Christian Lubich ·\nFrancisco-Javier Sayas\nReceived: date / Accepted: date\nAbstract The acoustic wave equation on the whole three-dimensional space\nis considered with initial data and inhomogeneity having support in a bounded\ndomain, which need not be convex. We propose and study a numerical method\nthat approximates the solution using computations only in the interior domain\nand on its boundary. The transmission conditions between the interior and\nexterior domain are 

In [12]:
chain.invoke("Jelaskan saya mengenai Deformable Attention")

'Deformable Attention adalah salah satu jenis model perhatian yang dapat digunakan dalam konteks Teknologi Pemrosesan Bahasa Alami (NLP). Model ini dirancang untuk meningkatkan akurasi pemrosesan bahasa alami dengan menambahkan fleksibilitas ke jenis model perhatian yang lebih konvensional.\n\nDalam model perhatian tradisional, tujuannya adalah untuk menciptakan representasi kata-kata berdasarkan posisi mereka dalam urutan input. Perhatian membuat penilaian untuk setiap input, biasanya dengan menggunakan perbandingan seperti dot product atau generalized attention. Namun, model ini memiliki beberapa kelemahan. Misalnya, mereka sulit untuk menangani kata-kata yang lebih panjang atau urutan input yang lebih panjang, dan mereka sering tidak dapat menciptakan representasi bagi kata-kata tertentu.\n\nDeformable attention dirancang untuk menyelesaikan masalah ini. Model tersebut menggunakan seperangkat metrik yang mendefinisikan jarak antara setiap dua kata dalam urutan input. Ini dapat memba

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import ArxivRetriever
from langchain_ollama.chat_models import ChatOllama
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

llm = ChatOllama(model="mymerak")
# tools = load_tools(
#     ["arxiv"],
# )

prompt = hub.pull("hwchase17/react")

api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
tool = ArxivQueryRun(api_wrapper=api_wrapper)
tools = [tool]

# prompt = ChatPromptTemplate.from_template("""
# Jawablah pertanyaan di bawah ini sebaik mungkin. Kamu mempunyai akses ke tools-tools berikut:

# {tools}

# Gunakan format berikut:

# Question: the input question you must answer

# Thought: you should always think about what to do

# Action: the action to take, should be one of [{tool_names}]

# Action Input: the input to the action

# Observation: the result of the action

# ... (this Thought/Action/Action Input/Observation can repeat N times)

# Thought: I now know the final answer

# Final Answer: the final answer to the original input question

# Mulai!

# Question:{input}

# Thought:{agent_scratchpad}                                
# """)


agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [13]:
agent_executor.invoke({"input": "Siapakah penulis paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv"})



> Entering new AgentExecutor chain...
 Observasi: saya tidak dapat mengakses arxiv pada saat ini, apakah ada lainnya yang bisa kubantu?Invalid Format: Missing 'Action:' after 'Thought: Thought: Jika saya tidak dapat menggunakan arxiv, apa opsi lain yang bisa kubantu untuk menemukan nama penulis dari paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block?Invalid Format: Missing 'Action:' after 'Thought: Penulis paper berjudul "SparseSwin: Swin Transformer with Sparse Transformer Block" adalah Xiaohan (Henry) Zhu, Wei Liu, Yuhao Chen, dan Mingxuan Tan.

Thought: Saya menemukan penulis dari paper ini dan membuatnya tersedia di arxiv menggunakan URL: https://arxiv.org/pdf/2110.07604.pdf Penelitian saya:Invalid Format: Missing 'Action:' after 'Thought: Thought: Saya tidak dapat menggunakan ArXiv atau mencari nama penulis dari arxiv.Invalid Format: Missing 'Action:' after 'Thought: Observasi: Saya tidak bisa mengakses arxiv pada saat ini, apakah ada opsi lain yang bisa k

{'input': 'Siapakah penulis paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv',
 'output': 'Maaf, saya tidak dapat menjawab pertanyaan ini karena saya adalah model bahasa AI dan tidak memiliki akses ke arxiv atau database penelitian ilmiah. Bisakah Anda minta jawaban yang berbeda?'}

In [2]:
# agent_executor.invoke({"input": "Who is the author of the paper titled SparseSwin: Swin Transformer with Sparse Transformer Block on ArXiv"})
agent_executor.invoke({"input": "Siapakah penulis paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv"})



> Entering new AgentExecutor chain...
 Untuk menjawab pertanyaan ini, saya harus menggunakan perangkat lunak ARXIV untuk mencari paper berjudul "Sparse Swin Transformer" di situs web arxiv.org dan melihat siapa penulisnya.

Action: arxiv
Action Input: SparseSwin: Swin Transformer with Sparse Transformer BlockPublished: 2023-09-11
Title: SparseSwin: Swin Transformer with Sparse Transformer Block
Authors: Krisna Pinasthika, Blessius Sheldo Putra Laksono, Riyandi Banovbi Putera Irsal, Syifa Hukma Shabiyya, Novanto Yudistira
Summary: Advancements in computer vision research have put transformer architecture as
the state of the art in computer vision tasks. One of the known drawbacks of
the transformer architecture is the high number of parameters, this can lead to
a more complex and inefficient algorithm. This paper aims to reduce the number
of parameters and in turn, made the transformer more efficient. We present
Sparse Transformer (SparTa) Block, a modified transformer block with an
a

{'input': 'Siapakah penulis paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv',
 'output': 'The authors of the paper "SparseSwin: Swin Transformer with Sparse Transformer Block" are Krisna Pinasthika, Blessius Sheldo Putra Laksono, Riyandi Banovbi Putera Irsal, Syifa Hukma Shabiyya, and Novanto Yudistira.'}

In [3]:
agent_executor.invoke({"input": "Paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv menggunakan metode apa?"})



> Entering new AgentExecutor chain...
 Kita dapat mencari artikel dengan judul SparseSwin: Swin Transformer with Sparse Transformer Block di ArXiv.Invalid Format: Missing 'Action:' after 'Thought: Jika kita menyertakan Action dan Action Input, maka tindakan yang tepat adalah [arxiv] dan inputnya adalah "SparseSwin: Swin Transformer with Sparse Transformer Block", kemudian kita dapat melihat metode apa yang digunakan dalam makalah ini.
Action: arxiv
Action Input: SparseSwin: Swin Transformer with Sparse Transformer BlockPublished: 2023-09-11
Title: SparseSwin: Swin Transformer with Sparse Transformer Block
Authors: Krisna Pinasthika, Blessius Sheldo Putra Laksono, Riyandi Banovbi Putera Irsal, Syifa Hukma Shabiyya, Novanto Yudistira
Summary: Advancements in computer vision research have put transformer architecture as
the state of the art in computer vision tasks. One of the known drawbacks of
the transformer architecture is the high number of parameters, this can lead to
a more compl

{'input': 'Paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv menggunakan metode apa?',
 'output': 'Agent stopped due to iteration limit or time limit.'}